In [1]:
import requests

def make_api_call(base_url, endpoint="", method="GET", **kwargs):
    # Construct the full URL
    full_url = f'{base_url}{endpoint}'

    # Make the API call
    response = requests.request(method=method, url=full_url, **kwargs)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response
    else:
        # If the request was not successful, raise an exception with the error message
        raise Exception(f'API request failed with status code {response.status_code}: {response.text}')


In [2]:
import pandas as pd

def get_binance_historical_data(symbol, interval, start_date, end_date):
    
    # define basic parameters for call
    base_url = 'https://fapi.binance.com'
    endpoint = '/fapi/v1/klines'
    method = 'GET'
    
    # Set the start time parameter in the params dictionary
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1500,
        'startTime': start_date, # Start time in milliseconds
        'endTime' : end_date
    }


    # Make initial API call to get candles
    response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

    candles_data = []

    while params['endTime'] > params['startTime']:
    
        # Append the received candles to the list
        candles_data.extend(response.json())

        # Make the next API call
        response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

        # Update the start time for the next API call
        params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms

    
    # Wrap the candles data as a pandas DataFrame
    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
               'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    dtype={
    'open_time': 'datetime64[ms, Asia/Jerusalem]',
    'open': 'float64',
    'high': 'float64',
    'low': 'float64',
    'close': 'float64',
    'volume': 'float64',
    'close_time': 'datetime64[ms, Asia/Jerusalem]',
    'quote_asset_volume': 'float64',
    'number_of_trades': 'int64',
    'taker_buy_base_asset_volume': 'float64',
    'taker_buy_quote_asset_volume': 'float64',
    'ignore': 'float64'
    }
    
    df = pd.DataFrame(candles_data, columns=columns)
    df = df.astype(dtype)

    return df

C:\Users\Tzlil\AppData\Local\Temp\ipykernel_33844\318752140.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from datetime import datetime

symbol = 'BTCUSDT'
interval = '30m'
start_date = int(datetime(year=2023, month=1, day=1).timestamp() * 1000)
end_date = int(datetime(year=2024, month=1, day=1).timestamp() * 1000)


btcusdt_df = get_binance_historical_data(symbol, interval, start_date, end_date)
btcusdt_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,2023-01-01 00:29:59.999000+02:00,1.279086e+08,35451,3338.726,5.520766e+07,0.0
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,2023-01-01 00:59:59.999000+02:00,1.473757e+08,41907,3740.250,6.174233e+07,0.0
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,2023-01-01 01:29:59.999000+02:00,8.608995e+07,27411,2761.688,4.558437e+07,0.0
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,2023-01-01 01:59:59.999000+02:00,5.171514e+07,17078,1772.384,2.930818e+07,0.0
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,2023-01-01 02:29:59.999000+02:00,4.681919e+07,16561,1373.220,2.269722e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,2023-12-31 22:29:59.999000+02:00,1.091479e+08,28539,1367.677,5.833755e+07,0.0
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,2023-12-31 22:59:59.999000+02:00,9.251599e+07,27578,1098.223,4.678832e+07,0.0
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,2023-12-31 23:29:59.999000+02:00,6.265517e+07,20273,862.639,3.679818e+07,0.0
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,2023-12-31 23:59:59.999000+02:00,9.909701e+07,24200,1099.718,4.687127e+07,0.0


In [4]:
# Remove duplicates in the 'open_time' column
btcusdt_df = btcusdt_df.drop_duplicates(subset=['open_time'])

# Sort the DataFrame by the 'open_time' column from earliest to latest
btcusdt_df = btcusdt_df.sort_values(by='open_time')


In [5]:
import numpy as np
def calculate_ATR(df: pd.DataFrame, atr_length: int) -> pd.Series:

    tr_series = np.maximum(df['high'], df['close'].shift(1)) - np.minimum(df['low'], df['close'].shift(1))    
    
    atr_series = tr_series.rolling(atr_length).mean()
    
    return atr_series

btcusdt_df['ATR_sell'] = calculate_ATR(btcusdt_df, 1)
btcusdt_df['ATR_buy'] = calculate_ATR(btcusdt_df, 300)
df_from_row_300 = btcusdt_df.iloc[300:]
df_from_row_300

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,ATR_sell,ATR_buy
300,2023-01-07 06:00:00+02:00,16944.3,16945.0,16936.6,16936.6,1108.984,2023-01-07 06:29:59.999000+02:00,1.878720e+07,7407,453.779,7.687351e+06,0.0,8.4,30.521667
301,2023-01-07 06:30:00+02:00,16936.6,16936.7,16928.0,16930.0,1417.184,2023-01-07 06:59:59.999000+02:00,2.399664e+07,8892,638.782,1.081618e+07,0.0,8.7,30.254000
302,2023-01-07 07:00:00+02:00,16930.0,16933.2,16927.8,16929.9,1036.175,2023-01-07 07:29:59.999000+02:00,1.754290e+07,6704,538.692,9.120173e+06,0.0,5.4,30.133667
303,2023-01-07 07:30:00+02:00,16930.0,16930.0,16925.2,16926.9,1070.877,2023-01-07 07:59:59.999000+02:00,1.812805e+07,6972,498.462,8.438060e+06,0.0,4.8,30.048333
304,2023-01-07 08:00:00+02:00,16926.8,16926.9,16915.8,16922.5,2323.424,2023-01-07 08:29:59.999000+02:00,3.931265e+07,12033,1155.523,1.955093e+07,0.0,11.1,29.993667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,2023-12-31 22:29:59.999000+02:00,1.091479e+08,28539,1367.677,5.833755e+07,0.0,124.5,188.802333
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,2023-12-31 22:59:59.999000+02:00,9.251599e+07,27578,1098.223,4.678832e+07,0.0,119.2,188.845667
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,2023-12-31 23:29:59.999000+02:00,6.265517e+07,20273,862.639,3.679818e+07,0.0,130.3,188.851000
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,2023-12-31 23:59:59.999000+02:00,9.909701e+07,24200,1099.718,4.687127e+07,0.0,130.9,188.790667


In [6]:
t_fast = 27
t_slow = 50
btcusdt_df['EMA_fast'] = btcusdt_df['close'].ewm(span=t_fast, min_periods=t_fast, adjust=False).mean()
btcusdt_df['EMA_slow'] = btcusdt_df['close'].ewm(span=t_slow, min_periods=t_slow, adjust=False).mean()
btcusdt_df['MacdDiff'] = btcusdt_df['EMA_fast'] - btcusdt_df['EMA_slow']

btcusdt_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,ATR_sell,ATR_buy,EMA_fast,EMA_slow,MacdDiff
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,2023-01-01 00:29:59.999000+02:00,1.279086e+08,35451,3338.726,5.520766e+07,0.0,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,2023-01-01 00:59:59.999000+02:00,1.473757e+08,41907,3740.250,6.174233e+07,0.0,89.0,NaN,NaN,NaN,NaN
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,2023-01-01 01:29:59.999000+02:00,8.608995e+07,27411,2761.688,4.558437e+07,0.0,41.5,NaN,NaN,NaN,NaN
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,2023-01-01 01:59:59.999000+02:00,5.171514e+07,17078,1772.384,2.930818e+07,0.0,30.4,NaN,NaN,NaN,NaN
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,2023-01-01 02:29:59.999000+02:00,4.681919e+07,16561,1373.220,2.269722e+07,0.0,27.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,2023-12-31 22:29:59.999000+02:00,1.091479e+08,28539,1367.677,5.833755e+07,0.0,124.5,188.802333,42561.766608,42481.625667,80.140941
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,2023-12-31 22:59:59.999000+02:00,9.251599e+07,27578,1098.223,4.678832e+07,0.0,119.2,188.845667,42564.433279,42486.232503,78.200776
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,2023-12-31 23:29:59.999000+02:00,6.265517e+07,20273,862.639,3.679818e+07,0.0,130.3,188.851000,42572.316616,42493.627307,78.689309
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,2023-12-31 23:59:59.999000+02:00,9.909701e+07,24200,1099.718,4.687127e+07,0.0,130.9,188.790667,42571.358286,42496.187020,75.171266


In [7]:

import numpy as np

def rolling_min(series: pd.Series, window):
    return series.rolling(window=window).min()

def rolling_max(series: pd.Series, window):
    return series.rolling(window=window).max()

def forward_fill(series: pd.Series):
    return series.ffill()

def SmoothSrs(srs, smoothing_f):
    smoothed_srs = np.zeros_like(srs)
    smoothed_srs[0] = srs[0]

    for i in range(1, len(srs)):
        if np.isnan(smoothed_srs[i - 1]):
            smoothed_srs[i] = srs[i]
        else:
            smoothed_srs[i] = smoothed_srs[i - 1] + smoothing_f * (srs[i] - smoothed_srs[i - 1])
    
    return smoothed_srs

def NormalizeSmoothSrs(series: pd.Series, window_length, smoothing_f):
    lowest = rolling_min(pd.Series(series), window_length)
    highest_range = rolling_max(pd.Series(series), window_length) - lowest

    normalized_series = (series - lowest) / highest_range * 100
    normalized_series = np.where(highest_range > 0, normalized_series, np.nan)
    normalized_series = forward_fill(pd.Series(normalized_series))

    smoothed_series = SmoothSrs(normalized_series, smoothing_f)
    return smoothed_series

def STC(df: pd.DataFrame, stc_length, smoothing_factor=0.5):
    
    normalized_macd = NormalizeSmoothSrs(df['MacdDiff'], stc_length, smoothing_factor)
    final_stc = NormalizeSmoothSrs(normalized_macd, stc_length, smoothing_factor)
    return final_stc

In [8]:

stc_series = STC(btcusdt_df, stc_length=80, smoothing_factor=0.5)
btcusdt_df['STC'] = stc_series
selected_columns = ['open_time', 'open', 'high', 'low', 'close', 'volume',
                    'ATR_buy', 'ATR_sell', 'MacdDiff', 'EMA_fast', 'EMA_slow',
                    'STC']

btcusdt_df_with_ta = btcusdt_df[selected_columns]
btcusdt_df_with_ta



,open_time,open,high,low,close,volume,ATR_buy,ATR_sell,MacdDiff,EMA_fast,EMA_slow,STC
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,NaN,89.0,NaN,NaN,NaN,NaN
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,NaN,41.5,NaN,NaN,NaN,NaN
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,NaN,30.4,NaN,NaN,NaN,NaN
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,NaN,27.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,188.802333,124.5,80.140941,42561.766608,42481.625667,98.543187
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,188.845667,119.2,78.200776,42564.433279,42486.232503,98.562249
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,188.851000,130.3,78.689309,42572.316616,42493.627307,98.528401
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,188.790667,130.9,75.171266,42571.358286,42496.187020,98.060646


In [9]:
def xATRTrailingStop_func(close, prev_close, prev_trailing_stop, loss_thr):
    if close > prev_trailing_stop and prev_close > prev_trailing_stop:
        return max(prev_trailing_stop, close - loss_thr)
    elif close < prev_trailing_stop and prev_close < prev_trailing_stop:
        return min(prev_trailing_stop, close + loss_thr)
    elif close > prev_trailing_stop:
        return close - loss_thr
    else:
        return close + loss_thr
 
# Filling ATRTrailingStop Variable
key_value = 2
btcusdt_df_with_ta["ATRTrailingStop_buy"] = [0.0] + [np.nan for i in range(len(btcusdt_df_with_ta) - 1)]
btcusdt_df_with_ta["ATRTrailingStop_sell"] = [0.0] + [np.nan for i in range(len(btcusdt_df_with_ta) - 1)]

btcusdt_df_with_ta['loss_threshold_buy'] = key_value * btcusdt_df_with_ta['ATR_buy']
btcusdt_df_with_ta['loss_threshold_sell'] = key_value * btcusdt_df_with_ta['ATR_sell']
print(len(btcusdt_df_with_ta))

for index, row in btcusdt_df_with_ta.iterrows():
    try:
        if index == 0:
            continue  # Skip the first row 
        btcusdt_df_with_ta.loc[index, "ATRTrailingStop_buy"] = xATRTrailingStop_func(
            row["close"],
            btcusdt_df_with_ta.loc[index - 1, "close"],
            btcusdt_df_with_ta.loc[index - 1, "ATRTrailingStop_buy"],
            row["loss_threshold_buy"],
        )
        btcusdt_df_with_ta.loc[index, "ATRTrailingStop_sell"] = xATRTrailingStop_func(
            row["close"],
            btcusdt_df_with_ta.loc[index - 1, "close"],
            btcusdt_df_with_ta.loc[index - 1, "ATRTrailingStop_sell"],
            row["loss_threshold_sell"],
        )
    except KeyError as e:
        print(f"KeyError at index {index}: {e}")

btcusdt_df_with_ta

C:\Users\Tzlil\AppData\Local\Temp\ipykernel_33844\1231398579.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcusdt_df_with_ta["ATRTrailingStop_buy"] = [0.0] + [np.nan for i in range(len(btcusdt_df_with_ta) - 1)]
C:\Users\Tzlil\AppData\Local\Temp\ipykernel_33844\1231398579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcusdt_df_with_ta["ATRTrailingStop_sell"] = [0.0] + [np.nan for i in range(len(btcusdt_df_with_ta) - 1)]
C:\Users\Tzlil\AppData\Local\Temp\ipykernel_33844\1231398579.py:16: Setti

17521
KeyError at index 3000: 2999


KeyError at index 6000: 5999
KeyError at index 9000: 8999
KeyError at index 12000: 11999
KeyError at index 15000: 14999
KeyError at index 18000: 17999
KeyError at index 21000: 20999
KeyError at index 24000: 23999
KeyError at index 27000: 26999
KeyError at index 30000: 29999
KeyError at index 33000: 32999


,open_time,open,high,low,close,volume,ATR_buy,ATR_sell,MacdDiff,EMA_fast,EMA_slow,STC,ATRTrailingStop_buy,ATRTrailingStop_sell,loss_threshold_buy,loss_threshold_sell
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,NaN,NaN
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,NaN,89.0,NaN,NaN,NaN,NaN,0.000,16337.1,NaN,178.0
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,NaN,41.5,NaN,NaN,NaN,NaN,0.000,16437.9,NaN,83.0
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,NaN,30.4,NaN,NaN,NaN,NaN,0.000,16476.8,NaN,60.8
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,NaN,27.5,NaN,NaN,NaN,NaN,0.000,16484.4,NaN,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,188.802333,124.5,80.140941,42561.766608,42481.625667,98.543187,42404.814,42508.5,377.604667,249.0
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,188.845667,119.2,78.200776,42564.433279,42486.232503,98.562249,42404.814,42508.5,377.691333,238.4
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,188.851000,130.3,78.689309,42572.316616,42493.627307,98.528401,42404.814,42508.5,377.702000,260.6
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,188.790667,130.9,75.171266,42571.358286,42496.187020,98.060646,42404.814,42508.5,377.581333,261.8


In [10]:
import vectorbt as vbt
# Calculating signals
ema = vbt.MA.run(btcusdt_df_with_ta["close"], 1, short_name='EMA', ewm=True)
 
btcusdt_df_with_ta["Above"] = ema.ma_crossed_above(btcusdt_df_with_ta["ATRTrailingStop_buy"])
btcusdt_df_with_ta["Below"] = ema.ma_crossed_below(btcusdt_df_with_ta["ATRTrailingStop_sell"])
 
btcusdt_df_with_ta["UT_Buy"] = (btcusdt_df_with_ta["close"] > btcusdt_df_with_ta["ATRTrailingStop_buy"]) & (btcusdt_df_with_ta["Above"]==True)
btcusdt_df_with_ta["UT_Sell"] = (btcusdt_df_with_ta["close"] < btcusdt_df_with_ta["ATRTrailingStop_sell"]) & (btcusdt_df_with_ta["Below"]==True)



C:\Users\Tzlil\AppData\Local\Temp\ipykernel_33844\1743828138.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcusdt_df_with_ta["Above"] = ema.ma_crossed_above(btcusdt_df_with_ta["ATRTrailingStop_buy"])


In [11]:

btcusdt_df_with_ta['BUY'] = (btcusdt_df_with_ta['UT_Buy'] & (btcusdt_df_with_ta['STC'] < 25) & (btcusdt_df_with_ta['STC'] < btcusdt_df_with_ta['STC'].shift(-1)))
btcusdt_df_with_ta['SELL'] = (btcusdt_df_with_ta['UT_Sell'] & (btcusdt_df_with_ta['STC'] > 75) & (btcusdt_df_with_ta['STC'] > btcusdt_df_with_ta['STC'].shift(-1)))
btcusdt_df_with_ta


,open_time,open,high,low,close,volume,ATR_buy,ATR_sell,MacdDiff,EMA_fast,...,ATRTrailingStop_buy,ATRTrailingStop_sell,loss_threshold_buy,loss_threshold_sell,Above,Below,UT_Buy,UT_Sell,BUY,SELL
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,NaN,NaN,NaN,NaN,...,0.000,0.0,NaN,NaN,False,False,False,False,False,False
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,NaN,89.0,NaN,NaN,...,0.000,16337.1,NaN,178.0,False,False,False,False,False,False
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,NaN,41.5,NaN,NaN,...,0.000,16437.9,NaN,83.0,False,False,False,False,False,False
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,NaN,30.4,NaN,NaN,...,0.000,16476.8,NaN,60.8,False,False,False,False,False,False
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,NaN,27.5,NaN,NaN,...,0.000,16484.4,NaN,55.0,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34016,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,188.802333,124.5,80.140941,42561.766608,...,42404.814,42508.5,377.604667,249.0,False,False,False,False,False,False
34017,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,188.845667,119.2,78.200776,42564.433279,...,42404.814,42508.5,377.691333,238.4,False,False,False,False,False,False
34018,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,188.851000,130.3,78.689309,42572.316616,...,42404.814,42508.5,377.702000,260.6,False,False,False,False,False,False
34019,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,188.790667,130.9,75.171266,42571.358286,...,42404.814,42508.5,377.581333,261.8,False,False,False,False,False,False


In [12]:
from enum import Enum

class ActionType(Enum):
    BUY = 1
    SELL = -1

# class BuyAndHoldStrategy():
#     def __init__(self):
#         pass
    
def calc_qty(real_price: float, curr_qty: float, balance: float, action: ActionType) -> float:
        
    if action == ActionType.BUY:
        if curr_qty >= 0: # Enter long
            qty = balance / real_price
        elif curr_qty < 0: # Close short
            qty = -1 * curr_qty
    
    elif action == ActionType.SELL:
        if curr_qty > 0: # Close long
            qty = curr_qty
        elif curr_qty <= 0: # Enter short
            qty =  balance / real_price 
    
    return qty

def calc_realistic_price(row: pd.Series ,action_type: ActionType, slippage_factor=np.inf):
    slippage_rate = ((row['close'] - row['open']) / row['open']) / slippage_factor
    slippage_price = row['open'] + row['open'] * slippage_rate
    
    if action_type == ActionType.BUY:
        return max(slippage_price, row['open'])
    else:
        return min(slippage_price, row['open'])  

def apply_strategy(row):
    strategy = 0
    if row['BUY'] == True:
        strategy = 1
    if row['SELL'] == True:
        strategy = -1
    return strategy
    
def basic_backtest(data: pd.DataFrame, balance: int, commission: float=0.0) -> pd.DataFrame:
    # Loop through the data to calculate portfolio value
    data.reset_index(inplace=True)
    
    # initialize
    data['qty'] = 0.0
    data['balance'] = 0.0
    data.loc[0,'balance'] = balance

    # implement strategy
    data['strategy'] = data.apply(apply_strategy, axis=1)
    
    for index, row in data.iterrows():
        curr_qty = data.loc[index - 1, 'qty'] if index > 0 else 0
        curr_balance = data.loc[index - 1, 'balance'] if index > 0 else balance

        if index == data.shape[0] - 1: # Close positions
            if curr_qty > 0:
                sell_price = calc_realistic_price(row, ActionType.SELL, slippage_factor=5.0)
                data.loc[index, 'balance'] = curr_balance + curr_qty * sell_price  - commission
                data.loc[index, 'qty'] = 0
            else:
                data.loc[index, 'balance'] = curr_balance 


        elif row['strategy'] == 1:  # Buy stocks
            # if i have 2 buys in a row copy the qty and skip 
            if curr_balance < 1:
                data.loc[index, 'qty'] = curr_qty
            else:
                buy_price = calc_realistic_price(row, ActionType.BUY, slippage_factor=5.0)
                qty_to_buy = calc_qty(buy_price, curr_qty, curr_balance, ActionType.BUY)
                data.loc[index, 'qty'] = curr_qty + qty_to_buy 
                data.loc[index, 'balance'] = curr_balance - qty_to_buy * buy_price - commission
            
        elif row['strategy'] == -1:  # Sell all stocks
            # if the first command is sell - ignore!!
            if (curr_balance == balance) & (curr_qty == 0):
                data.loc[index, 'balance'] = balance
            else: #sell all
                sell_price = calc_realistic_price(row, ActionType.SELL, slippage_factor=5.0)
                data.loc[index, 'balance'] = curr_balance + sell_price * curr_qty - commission
                data.loc[index, 'qty'] = 0
        else:
            if index > 0:
                data.loc[index, 'qty'] = curr_qty
                data.loc[index, 'balance'] = curr_balance

            else:
                data.loc[index, 'qty'] = 0 
        

    # Calculate portfolio value
    data['portfolio_value'] = data['close'] * data['qty'] + data['balance']
    return data



balance = 10000
b_df = basic_backtest(btcusdt_df_with_ta.copy(deep=True), balance)
b_df.drop(columns=['index'], axis=1, inplace=True)
b_df



,open_time,open,high,low,close,volume,ATR_buy,ATR_sell,MacdDiff,EMA_fast,...,Above,Below,UT_Buy,UT_Sell,BUY,SELL,qty,balance,strategy,portfolio_value
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,0.0,10000.000000,0,10000.000000
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,NaN,89.0,NaN,NaN,...,False,False,False,False,False,False,0.0,10000.000000,0,10000.000000
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,NaN,41.5,NaN,NaN,...,False,False,False,False,False,False,0.0,10000.000000,0,10000.000000
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,NaN,30.4,NaN,NaN,...,False,False,False,False,False,False,0.0,10000.000000,0,10000.000000
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,NaN,27.5,NaN,NaN,...,False,False,False,False,False,False,0.0,10000.000000,0,10000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17516,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,188.802333,124.5,80.140941,42561.766608,...,False,False,False,False,False,False,0.0,17593.392326,0,17593.392326
17517,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,188.845667,119.2,78.200776,42564.433279,...,False,False,False,False,False,False,0.0,17593.392326,0,17593.392326
17518,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,188.851000,130.3,78.689309,42572.316616,...,False,False,False,False,False,False,0.0,17593.392326,0,17593.392326
17519,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,188.790667,130.9,75.171266,42571.358286,...,False,False,False,False,False,False,0.0,17593.392326,0,17593.392326


In [13]:
# Import Plotly and pandas
import plotly.graph_objects as go
import pandas as pd
# Create a figure with OHLC trace
fig = go.Figure(data=go.Candlestick(x=b_df['open_time'], # Use open_date as x-axis values
                open=b_df['open'],
                high=b_df['high'],
                low=b_df['low'],
                close=b_df['close'],
                line_width=0.2))

# Add red arrows for action = -1
fig.add_trace(go.Scatter(x=b_df[b_df['strategy'] == -1]['open_time'], # Use open_date for x-axis
                         y=b_df[b_df['strategy'] == -1]['high'] + b_df[b_df['strategy'] == -1]['low'] / 10,
                         mode='markers',
                         marker_symbol='triangle-down',
                         marker_color='red',
                         marker_size=10,
                         name='Sell'))

# Add green arrows for action = 1
fig.add_trace(go.Scatter(x=b_df[b_df['strategy'] == 1]['open_time'], # Use open_date for x-axis
                         y=b_df[b_df['strategy'] == 1]['low'] - b_df[b_df['strategy'] == 1]['low'] / 10,
                         mode='markers',
                         marker_symbol='triangle-up',
                         marker_color='green',
                         marker_size=10,
                         name='Buy'))

# Customize the layout and the interactive controls
fig.update_layout(
    title='Chart with Buy and Sell Signals',
    yaxis_title='Price',
    xaxis_title='Date',
    hovermode='x unified', # Show the hover data for both traces
    margin=dict(l=20, r=20, t=50, b=20), # Reduce the white space around the plot
    dragmode='pan', # Change the default action to pan instead of zoom
    xaxis=dict( # Enable the rangeslider and set the rangebreaks
        rangeslider=dict(visible=True),
        rangebreaks=[
            dict(bounds=["sat", "sun"]), # Hide weekends
        ]
    )
)

# Add zoom buttons for both x and y axes
fig.update_xaxes(
    showspikes=True, # Show spike lines
    spikemode='across', # Show spike lines across the plot
    spikesnap='cursor', # Snap spike lines to the cursor
    showline=True, # Show the x-axis line
    linecolor='black', # Set the x-axis line color
    showgrid=False, # Hide the x-axis grid
    fixedrange=False, # Enable x-axis zoom
    rangeselector=dict( # Add buttons to select the x-axis range
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.update_yaxes(
    showspikes=True, # Show spike lines
    spikemode='across', # Show spike lines across the plot
    spikesnap='cursor', # Snap spike lines to the cursor
    showline=True, # Show the y-axis line
    linecolor='black', # Set the y-axis line color
    showgrid=True, # Show the y-axis grid
    gridcolor='lightgray', # Set the y-axis grid color
    fixedrange=False, # Enable y-axis zoom
    autorange = True,
)

cs = fig.data[0]

# Set line and fill colors
cs.increasing.fillcolor = '#3D9970'
cs.increasing.line.color = '#000000'
cs.decreasing.fillcolor = '#FF4136'
cs.decreasing.line.color = '#000000'

# Show the figure
fig.show()


In [14]:
b_df['profit_per'] = (b_df['portfolio_value'] -balance) / balance


In [15]:
import plotly.graph_objects as go

# Create a line plot
fig = go.Figure()

# Add a line trace for 'profit_per' against 'open_time'
fig.add_trace(go.Scatter(x=b_df['open_time'], y=b_df['profit_per'], mode='lines', name='Profit Percentage'))

# Update layout to customize axes labels
fig.update_layout(
    title='Profit Percentage Over Time',
    xaxis=dict(title='Open Time'), 
    yaxis=dict(title='Profit Percentage') 
)

# Show the plot
fig.show()


In [16]:
def calc_total_return(portfolio_values):
    return (portfolio_values.iloc[-1] / portfolio_values.iloc[0]) - 1.0

def calc_annualized_return(portfolio_values):
    yearly_trading_days = 252
    portfolio_trading_days = portfolio_values.shape[0]
    portfolio_trading_years = portfolio_trading_days / yearly_trading_days 
    return (portfolio_values.iloc[-1] / portfolio_values.iloc[0])**(1/portfolio_trading_years) - 1.0

def calc_annualized_sharpe(portfolio_values: pd.Series, rf: float=0.0):
    yearly_trading_days = 252
    annualized_return = calc_annualized_return(portfolio_values)
    annualized_std = portfolio_values.pct_change().std() * np.sqrt(yearly_trading_days)
    if annualized_std is None or annualized_std == 0:
        return 0
    sharpe = (annualized_return - rf) / annualized_std
    return sharpe

def calc_downside_deviation(portfolio_values):
    porfolio_returns = portfolio_values.pct_change().dropna()
    return porfolio_returns[porfolio_returns < 0].std()

def calc_sortino(portfolio_values, rf=0.0):
    yearly_trading_days = 252
    down_deviation = calc_downside_deviation(portfolio_values) * np.sqrt(yearly_trading_days)
    annualized_return = calc_annualized_return(portfolio_values)
    if down_deviation is None or down_deviation == 0:
        return 0
    sortino = (annualized_return - rf) / down_deviation
    return sortino

def calc_max_drawdown(portfolio_values):
    cumulative_max = portfolio_values.cummax()
    drawdown = (cumulative_max - portfolio_values) / cumulative_max
    return drawdown.max()

def calc_calmar(portfolio_values):
    max_drawdown = calc_max_drawdown(portfolio_values)
    annualized_return = calc_annualized_return(portfolio_values)
    return annualized_return / max_drawdown

def evaluate_strategy(b_df, strat_name):
    total_return = calc_total_return(b_df['portfolio_value'])
    annualized_return = calc_annualized_return(b_df['portfolio_value'])
    annualized_sharpe = calc_annualized_sharpe(b_df['portfolio_value'])
    sortino_ratio = calc_sortino(b_df['portfolio_value'])
    max_drawdown = calc_max_drawdown(b_df['portfolio_value'])
    calmar_ratio = calc_calmar(b_df['portfolio_value'])
    
    print(f"Results for {strat_name}:")
    print(f"Total Return: {total_return:.2%}")
    print(f"Annualized Return: {annualized_return:.2%}")
    print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
    print(f"Sortino Ratio: {sortino_ratio:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2%}")
    print(f"Calmar Ratio: {calmar_ratio:.2f}")

evaluate_strategy(b_df, 'UTBOT with STC')


Results for UTBOT with STC:
Total Return: 75.93%
Annualized Return: 0.82%
Annualized Sharpe Ratio: 0.26
Sortino Ratio: 0.21
Max Drawdown: 15.81%
Calmar Ratio: 0.05
